In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import sodshock

## Obtain solution from exact Riemann solver

In [2]:
gamma = 1.4
dustFrac = 0.0
npts = 1000
t = 0.2
left_state = (1,1,0)
right_state = (0.1, 0.125, 0.)

positions, regions, exact_solution = sodshock.solve(left_state=left_state, \
    right_state=right_state, geometry=(0., 1., 0.5), t=t, 
    gamma=gamma, npts=npts, dustFrac=dustFrac)

## Find latest solution file and plot its solution

In [ ]:
# get all filenames ending in *.csv within the solnData folder
filenames_raw = [f for f in os.listdir('solnData') if f.endswith('.csv')]
print('num solutions:', len(filenames_raw))

# remove the .csv extension
filenames = [f[:-4] for f in filenames_raw]

# remove everything up to the third underscore
filenames = [f[f.rfind('_') + 1:] for f in filenames]

# convert strings into integers
filenames = [int(f) for f in filenames]

# get largest value in filenames
max_value = max(filenames)

# get sub-string of the first element in filenames_raw going from the first character to the third underscore
sub_string = filenames_raw[0][:filenames_raw[0].rfind('_') + 1:]

latest_solution = f'{sub_string}{max_value:06}.csv'
print('last solution file:', latest_solution)

df = pd.read_csv(f'solnData/{latest_solution}')

# create reduced data frame to plot markers at consistent intervals (i.e. every 0.025 x-units)
number_of_rows = len(df)
interval_x_axis = 0.025
interval = int(number_of_rows * interval_x_axis)
indices = np.arange(len(df)) % interval == 0
df_reduced =df.iloc[indices]

# plot solution for each variable using subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=("density", "velocity", "pressure"))
fig.add_trace(go.Scatter(x=df['x'], y=df['rho'], name='density', mode='lines', line=dict(color=px.colors.qualitative.Plotly[0])), row=1, col=1)
fig.add_trace(go.Scatter(x=df_reduced['x'], y=df_reduced['rho'], name='density', mode='markers', marker=dict(size=10, symbol='circle-open', color=px.colors.qualitative.Plotly[0])), row=1, col=1)
fig.add_trace(go.Scatter(x=exact_solution['x'], y=exact_solution['rho'], name='exact density', line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['x'], y=df['u'], name='velocity', mode='lines', line=dict(color=px.colors.qualitative.Plotly[2])), row=1, col=2)
fig.add_trace(go.Scatter(x=df_reduced['x'], y=df_reduced['u'], name='velocity', mode='markers', marker=dict(size=10, symbol='circle-open', color=px.colors.qualitative.Plotly[2])), row=1, col=2)
fig.add_trace(go.Scatter(x=exact_solution['x'], y=exact_solution['u'], name='exact velocity', line=dict(color='black')), row=1, col=2)

fig.add_trace(go.Scatter(x=df['x'], y=df['p'], name='pressure', mode='lines', line=dict(color=px.colors.qualitative.Plotly[6])), row=1, col=3)
fig.add_trace(go.Scatter(x=df_reduced['x'], y=df_reduced['p'], name='pressure', mode='markers', marker=dict(size=10, symbol='circle-open', color=px.colors.qualitative.Plotly[6])), row=1, col=3)
fig.add_trace(go.Scatter(x=exact_solution['x'], y=exact_solution['p'], name='exact pressure', line=dict(color='black')), row=1, col=3)

fig.update_layout(font_size=20, height=400, width=1200, showlegend=False)
fig.update_annotations(font_size=24)
fig.show()

num solutions: 391


## Animate entire solution

In [5]:


# animation speed (in frames per second, fps)
fps = 60

# get all filenames ending in *.csv within the solnData folder
filenames_raw = [f for f in os.listdir('solnData') if f.endswith('.csv')]
filenames_raw.sort()

# read first latest_solution and plot
df = pd.read_csv(f'solnData/{filenames_raw[0]}')
fig = go.Figure(data=[go.Scatter(x=df['x'], y=df['rho'], name='rho', mode='lines'),
                     go.Scatter(x=df['x'], y=df['u'], name='u', mode='lines'),
                     go.Scatter(x=df['x'], y=df['p'], name='p', mode='lines')])
fig.update_layout(template='plotly_dark')

frames = list()
for filename in filenames_raw[1:]:
    df = pd.read_csv(f'solnData/{filename}')
    frames.append(go.Frame(data=[go.Scatter(x=df['x'], y=df['rho'], name='rho', mode='lines'),
                                go.Scatter(x=df['x'], y=df['u'], name='u', mode='lines'),
                                go.Scatter(x=df['x'], y=df['p'], name='p', mode='lines')]))

fig.update(frames=frames)
fig.update_layout(updatemenus=[dict(type="buttons", showactive=False, buttons=[dict(label="Play", method="animate",
    args=[None, {"frame": {"duration": 1000/fps, "redraw": False},
    "fromcurrent": True, "transition": {"duration": 300}}]), 
    dict(label="Pause", method="animate",
    args=[[None], {"frame": {"duration": 0, "redraw": False},
    "mode": "immediate", "transition": {"duration": 0}}])]
    )])
fig.show()